In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from category_encoders import CatBoostEncoder

In [ ]:
df = pd.read_csv("ramen_ratings.csv")

In [ ]:
df.set_index("Review #", inplace=True)
df = df.dropna(subset=["Style", "Country", "Brand"])

# Top Ten column will show whether the item is in top ten
df.loc[df["Top Ten"].notna(), "Top Ten"] = 1
df.loc[df["Top Ten"].isna(), "Top Ten"] = 0

# Drop unrated rows and change data types
df = df[df["Stars"] != "Unrated"]
df["Stars"] = df["Stars"].astype(float)
df["Top Ten"] = df["Top Ten"].astype(int)

df.info()
df.head()

In [ ]:
X = df[["Brand", "Variety", "Country", "Top Ten", "Style"]]
y = df["Stars"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
catboost_cols = ["Brand", "Variety", "Country"]
onehot_cols = ["Style"]

In [ ]:
catboost_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("encoder", CatBoostEncoder()),
    ]
)

onehot_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    [
        ("catboost", catboost_pipeline, catboost_cols),
        ("onehot", onehot_pipeline, onehot_cols),
        ("passthrough", "passthrough", ["Top Ten"]),
    ]
)

model = Pipeline(
    [("preprocessing", preprocessor), ("model", RandomForestRegressor(random_state=42))]
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

error = mean_squared_error(y_test, y_pred) ** (1 / 2)
error

In [ ]:
# Calculate the importance of features
feature_names = model.named_steps["preprocessing"].get_feature_names_out()

importances = model.named_steps["model"].feature_importances_

feat_importance_df = pd.DataFrame(
    {"Feature": feature_names, "Importance": importances}
).sort_values(by="Importance", ascending=False)

feat_importance_df

In [ ]:
import boto3

# Replace with your actual credentials and info
bucket_name = "dataminds-homeworks"
s3_file_key = "nihat-mammadli-fe2.ipynb"
local_file_path = "nihat-mammadli-fe2.ipynb"

# Create an S3 client
s3 = boto3.client("s3")

# Upload the file
try:
    s3.upload_file(local_file_path, bucket_name, s3_file_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_file_key}")
except Exception as e:
    print("Error uploading file:", e)